In [ ]:
import gurobipy as gp
import math
import pandas as pd

In [ ]:
IP = 0
MIP = 1
LP = 2


# model for Karatsuba multiplication algorithm
def model_karatsuba(N, model_type, m):
    # find smallest integer k such that N <= 2^k
    k = 0
    num = 1
    while num < N:
        num *= 2
        k += 1
    
    bits0 = math.ceil(k/2)
    bits1 = math.floor(k/2)
    bits2 = math.ceil(k/2) + 1
        
        
    # Add variables
    if model_type == IP:
        X0 = m.addVar(vtype=gp.GRB.INTEGER, name="X0")
        Y0 = m.addVar(vtype=gp.GRB.INTEGER, name="Y0")
        Y0_bits = m.addVars(1, bits0, vtype=gp.GRB.BINARY, name="Y0_bits")
        Z0 = m.addVar(vtype=gp.GRB.INTEGER, name="Z0")
        Z0_products = m.addVars(1, bits0, vtype=gp.GRB.INTEGER, name="Z0_products")
        
        X1 = m.addVar(vtype=gp.GRB.INTEGER, name="X1")
        Y1 = m.addVar(vtype=gp.GRB.INTEGER, name="Y1")
        Y1_bits = m.addVars(1, bits1, vtype=gp.GRB.BINARY, name="Y1_bits")
        Z1 = m.addVar(vtype=gp.GRB.INTEGER, name="Z1")
        Z1_products = m.addVars(1, bits1, vtype=gp.GRB.INTEGER, name="Z1_products")
        
        X2 = m.addVar(vtype=gp.GRB.INTEGER, name="X2")
        Y2 = m.addVar(vtype=gp.GRB.INTEGER, name="Y2")
        Y2_bits = m.addVars(1, bits2, vtype=gp.GRB.BINARY, name="Y2_bits")
        Z2 = m.addVar(vtype=gp.GRB.INTEGER, name="Z2")
        Z2_products = m.addVars(1, bits2, vtype=gp.GRB.INTEGER, name="Z2_products")
        
    elif model_type == MIP or model_type == LP:
        X0 = m.addVar(vtype=gp.GRB.CONTINUOUS, name="X0")
        Y0 = m.addVar(vtype=gp.GRB.CONTINUOUS, name="Y0")
        Z0 = m.addVar(vtype=gp.GRB.CONTINUOUS, name="Z0")
        Z0_products = m.addVars(1, bits0, vtype=gp.GRB.CONTINUOUS, name="Z0_products")
        
        X1 = m.addVar(vtype=gp.GRB.CONTINUOUS, name="X1")
        Y1 = m.addVar(vtype=gp.GRB.CONTINUOUS, name="Y1")
        Z1 = m.addVar(vtype=gp.GRB.CONTINUOUS, name="Z1")
        Z1_products = m.addVars(1, bits1, vtype=gp.GRB.CONTINUOUS, name="Z1_products")
        
        X2 = m.addVar(vtype=gp.GRB.CONTINUOUS, name="X2")
        Y2 = m.addVar(vtype=gp.GRB.CONTINUOUS, name="Y2")
        Z2 = m.addVar(vtype=gp.GRB.CONTINUOUS, name="Z2")
        Z2_products = m.addVars(1, bits2, vtype=gp.GRB.CONTINUOUS, name="Z2_products")
        
        if model_type == MIP:
            Y0_bits = m.addVars(1, bits0, vtype=gp.GRB.BINARY, name="Y0_bits")
            Y1_bits = m.addVars(1, bits1, vtype=gp.GRB.BINARY, name="Y1_bits")
            Y2_bits = m.addVars(1, bits2, vtype=gp.GRB.BINARY, name="Y2_bits")
        else:
            Y0_bits = m.addVars(1, bits0, vtype=gp.GRB.CONTINUOUS, name="Y0_bits")
            Y1_bits = m.addVars(1, bits1, vtype=gp.GRB.CONTINUOUS, name="Y1_bits")
            Y2_bits = m.addVars(1, bits2, vtype=gp.GRB.CONTINUOUS, name="Y2_bits")

        
    # Set objective
    m.setObjective(0, gp.GRB.MAXIMIZE)

    
    # Add constraints
    
    # add main constraint
    m.addConstr(N == Z1*(2**(2*bits0) - 2**(bits0)) + Z2*2**bits0 + Z0*(1 - 2**bits0))
    
    # add constraints for Z0 and Y0
    power = 1
    Y0_sum = 0
    Z0_sum = 0
    for i in range(bits0):
        Y0_sum += power * Y0_bits[0,i]
        Z0_sum += power * Z0_products[0,i]
        power *= 2
    
    m.addConstr(Y0 == Y0_sum)
    m.addConstr(Z0 == Z0_sum)
    
    for i in range(bits0):
        m.addConstr(Z0_products[0,i] <= N * Y0_bits[0,i])
        m.addConstr(Z0_products[0,i] <= X0)
        m.addConstr(X0 - N*(1 - Y0_bits[0,i]) <= Z0_products[0,i])
        m.addConstr(0 <= Z0_products[0,i])
    
    # add constraints for Z1 and Y1
    power = 1
    Y1_sum = 0
    Z1_sum = 0
    for i in range(bits1):
        Y1_sum += power * Y1_bits[0,i]
        Z1_sum += power * Z1_products[0,i]
        power *= 2
    
    m.addConstr(Y1 == Y1_sum)
    m.addConstr(Z1 == Z1_sum)
    
    for i in range(bits1):
        m.addConstr(Z1_products[0,i] <= N * Y1_bits[0,i])
        m.addConstr(Z1_products[0,i] <= X1)
        m.addConstr(X1 - N*(1 - Y1_bits[0,i]) <= Z1_products[0,i])
        m.addConstr(0 <= Z1_products[0,i])
    
    # constraints for Z2
    power = 1
    Y2_sum = 0
    Z2_sum = 0
    for i in range(bits2):
        Y2_sum += power * Y2_bits[0,i]
        Z2_sum += power * Z2_products[0,i]
        power *= 2
    
    m.addConstr(Y2 == Y2_sum)
    m.addConstr(Z2 == Z2_sum)
    
    for i in range(bits2):
        m.addConstr(Z2_products[0,i] <= N * Y2_bits[0,i])
        m.addConstr(Z2_products[0,i] <= X2)
        m.addConstr(X2 - N*(1 - Y2_bits[0,i]) <= Z2_products[0,i])
        m.addConstr(0 <= Z2_products[0,i])
        
    # constraints for making X2 = X1 + X0 and Y2 = Y1 + Y0
    m.addConstr(X2 == X1 + X0)
    m.addConstr(Y2 == Y1 + Y0)
    
    # constraints to prevent trivial factorization
    m.addConstr(X1*2**bits0 + X0 <= math.floor(N/2))
    m.addConstr(2 <= X1*2**bits0 + X0)
    
    
    # extra constraints for LP relaxation
    if model_type == LP:
        for i in range(bits0):
            m.addConstr(0 <= Y0_bits[0,i])
            m.addConstr(Y0_bits[0,i] <= 1)
        
        for i in range(bits1):
            m.addConstr(0 <= Y1_bits[0,i])
            m.addConstr(Y1_bits[0,i] <= 1)
        
        for i in range(bits2):
            m.addConstr(0 <= Y2_bits[0,i])
            m.addConstr(Y2_bits[0,i] <= 1)
    
    
    return bits0
    

In [ ]:
current_type = IP
start = 1000000
end = start + 300
data = []

for N in range(start, end+1):
    m = gp.Model()
    m.Params.LogToConsole = 0
    m.Params.Threads = 2
    
    expt = model_karatsuba(N, current_type, m)
    
    for i in range(10):
        m.reset(1)
        m.optimize()

        if m.status == gp.GRB.OPTIMAL:
            # get values of X and Y
            factorX = m.getVarByName("X1").x * 2**expt + m.getVarByName("X0").x
            factorY = m.getVarByName("Y1").x * 2**expt + m.getVarByName("Y0").x
            
            correct = 0
            if factorX.is_integer() and factorY.is_integer() \
               and N == factorX * factorY:
                correct = 1


            data.append([N, factorX, factorY, correct, \
                         m.NumVars, m.NumIntVars, m.NumBinVars, m.NumConstrs, \
                         m.NodeCount, m.IterCount, m.runtime, m.IsMIP])
        elif m.status == gp.GRB.INFEASIBLE:
            data.append([N, "INFEASIBLE", "INFEASIBLE", "N/A", \
                         m.NumVars, m.NumIntVars, m.NumBinVars, m.NumConstrs, \
                         m.NodeCount, m.IterCount, m.runtime, m.IsMIP])

In [ ]:
df = pd.DataFrame(data, columns=["N", "Factor X", "Factor Y", "Is Correct", \
                                 "Num Vars", "Num Integer Vars", "Num Binary Vars", "Num Constraints", \
                                 "Node Count", "Simplex Iterations Count", "Runtime", "Is MIP"])

In [ ]:
print(df)